In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
from datetime import datetime,date,timedelta
import pickle

In [2]:
ordir={"North":0.0,
    "NNE":22.5,
    "NE": 45.0,
    "ENE":67.5,
    "East":90.0,
    "ESE":112.5,
    "SE":135.0,
    "SSE":157.5,
    "South":180,
    "SSW":202.5,
    "SW":225.0,
    "WSW":247.5,
    "West":270.0,
    "WNW":292.5,
    "NW":315.0,
    "NNW":337.5}

In [3]:
def dailywind(d):
    wdate = d
    url = f"https://www.wunderground.com/dashboard/pws/KCASANTA3697/table/{wdate}/{wdate}/daily"
    html = urlopen(url)
    soup = BeautifulSoup(html)

    headers = [th.getText() for th in soup.findAll('thead')[3].findAll('th')]
    rows = soup.findAll('tbody')[3].findAll('tr')
    hourly_data = [[td.getText() for td in rows[i].findAll('td')] for i in range(len(rows))]

    df_weather = pd.DataFrame(hourly_data,columns=headers)
    df_wind = df_weather[['Time','Temperature','Wind','Speed','Gust']]
    df_wind['Date'] = wdate
    df_wind['Azimuth']= df_wind['Wind'].map(ordir)

    df_wind = df_wind[['Date','Time','Temperature','Wind','Azimuth','Speed','Gust']]
    df_wind.columns = ['Date', 'Time','Temp','Wind','Azimuth', 'Speed', 'Gust']

    return df_wind

In [4]:
now = datetime.now().date()
sdate= datetime(2021,6,1).date()
datediff = (now-sdate).days
print(f"# full days to pull between start date ({sdate}) and now ({now}): {datediff}")

# full days to pull between start date (2021-06-01) and now (2022-10-24): 510


In [5]:
nested_dfs=[]
for i in range(datediff):
    weatherdate = sdate + timedelta(days = i)
    print(weatherdate)
    nested_dfs.append(dailywind(weatherdate))

2021-06-01
2021-06-02
2021-06-03
2021-06-04
2021-06-05
2021-06-06
2021-06-07
2021-06-08
2021-06-09
2021-06-10
2021-06-11
2021-06-12
2021-06-13
2021-06-14
2021-06-15
2021-06-16
2021-06-17
2021-06-18
2021-06-19
2021-06-20
2021-06-21
2021-06-22
2021-06-23
2021-06-24
2021-06-25
2021-06-26
2021-06-27
2021-06-28
2021-06-29
2021-06-30
2021-07-01
2021-07-02
2021-07-03
2021-07-04
2021-07-05
2021-07-06
2021-07-07
2021-07-08
2021-07-09
2021-07-10
2021-07-11
2021-07-12
2021-07-13
2021-07-14
2021-07-15
2021-07-16
2021-07-17
2021-07-18
2021-07-19
2021-07-20
2021-07-21
2021-07-22
2021-07-23
2021-07-24
2021-07-25
2021-07-26
2021-07-27
2021-07-28
2021-07-29
2021-07-30
2021-07-31
2021-08-01
2021-08-02
2021-08-03
2021-08-04
2021-08-05
2021-08-06
2021-08-07
2021-08-08
2021-08-09
2021-08-10
2021-08-11
2021-08-12
2021-08-13
2021-08-14
2021-08-15
2021-08-16
2021-08-17
2021-08-18
2021-08-19
2021-08-20
2021-08-21
2021-08-22
2021-08-23
2021-08-24
2021-08-25
2021-08-26
2021-08-27
2021-08-28
2021-08-29
2021-08-30

HTTPError: HTTP Error 404: Not Found

In [6]:
sdatenew=nested_dfs[-1].Date.iloc[0]+timedelta(days=1)
datediffnew=(now-sdatenew).days
print(f"interrupted before pulling: {sdatenew}")
print(f"days pulled: {len(nested_dfs)}")
print(f"days remaining: {datediffnew}")

interrupted before pulling: 2022-09-07
days pulled: 463
days remaining: 47


In [7]:
for i in range(datediffnew):
    weatherdate = sdatenew + timedelta(days = i)
    print(weatherdate)
    nested_dfs.append(dailywind(weatherdate))

HTTPError: HTTP Error 404: Not Found

In [8]:
sdatenew2=nested_dfs[-1].Date.iloc[0]+timedelta(days=1)
datediffnew2=(now-sdatenew2).days
print(f"interrupted before pulling: {sdatenew2}")
print(f"days pulled: {len(nested_dfs)}")
print(f"days remaining: {datediffnew2}")

interrupted before pulling: 2022-09-25
days pulled: 481
days remaining: 29


In [9]:
for i in range(datediffnew2):
    weatherdate = sdatenew2 + timedelta(days = i)
    print(weatherdate)
    nested_dfs.append(dailywind(weatherdate))

2022-09-25
2022-09-26
2022-09-27
2022-09-28
2022-09-29
2022-09-30
2022-10-01
2022-10-02
2022-10-03
2022-10-04
2022-10-05
2022-10-06
2022-10-07
2022-10-08
2022-10-09
2022-10-10
2022-10-11
2022-10-12
2022-10-13
2022-10-14
2022-10-15
2022-10-16
2022-10-17
2022-10-18
2022-10-19
2022-10-20
2022-10-21
2022-10-22
2022-10-23


In [16]:
allwind = nested_dfs[0]
for i in range(1,len(nested_dfs)):
    allwind = pd.concat([allwind,nested_dfs[i]],axis=0)
allwind['Speed'] = allwind['Speed'].str[:-5].astype(float)
allwind['Gust'][allwind['Gust']=='--'] = '0.0 °mph'
allwind['Gust'] = allwind['Gust'].str[:-5].astype(float)
allwind['Temp'] = allwind['Temp'].str[:-3].astype(float)
allwind['Month'] = pd.DatetimeIndex(allwind['Date']).month_name().str[:3]

In [17]:
allwind.head()

,Date,Time,Temp,Wind,Azimuth,Speed,Gust,Month
0,2021-06-01,12:04 AM,52.8,WSW,247.5,0.2,1.7,Jun
1,2021-06-01,12:09 AM,53.0,SW,225.0,0.1,1.4,Jun
2,2021-06-01,12:14 AM,53.0,SSW,202.5,0.7,2.0,Jun
3,2021-06-01,12:19 AM,52.8,SSE,157.5,0.1,1.7,Jun
4,2021-06-01,12:24 AM,52.8,SW,225.0,1.0,2.2,Jun


In [20]:
allwind.to_pickle(f'ppoint_raw_{sdate}_{now-timedelta(days=1)}.pkl')

In [21]:
allwind.to_csv(f'ppoint_raw_{sdate}_{now-timedelta(days=1)}.csv')